# 理论知识查漏补缺

## 判别模型与生成模型

- 判别模型：直接对$P(y|x)$建模
- 生成模型：判断$P(y=0|x)$和$P(y=1|x)$谁更大一点，借助贝叶斯公式，对$P(x,y)$联合概率进行建模

## 高斯判别分析

## 朴素贝叶斯

动机：简化运算



$$\hat{y} = arg\max\limits_{y}P(x|y)P(y)$$
### 对于先验P(y)
- 二分类：$y\sim Bernoulli$
- 多分类：$y\sim Categorial$

### 对于似然p(x|y)

假设：**条件独立性**（特征在类别确定的情况下是独立的）
$$P(x|y)=\prod_{j=1}^mP(x_j|y)$$
- x离散：$x_j\sim Categorial$
- x连续：$x_j \sim N(\mu_j,\sigma^2_j)$

## 对Naive Bayes的个人理解

- 是一种生成模型，利用贝叶斯公式，实际上学习的是联合分布，通过极大似然估计求解参数，计算后验概率。具有很强的假设：条件独立性（特征在类别确定的情况下是条件独立的）。

- P(X,y) =p(x|y)p(y)  = p(x1|y) p(x2|y) p(x3|y) p(y)
- 实际上求的参数就是条件概率和先验概率，其中有多少类就有多少个先验概率参数，条件概率参数与类别个数以及每个特征的特征值个数有关。
- 在训练集中求得各个参数，新数据来了之后进行“查表”，进行条件概率与先验概率的相乘，求得属于每一类的概率。
- 为了防止条件概率参数为0影响最后的结果，进行拉普拉斯平滑，通常做法是在每一类对应的特征的所有水平上加一。
- 优点是高效易于实现，缺点是分类性能不高

# python实现Naive Bayes

In [21]:
import pandas as pd
import numpy as np

In [7]:
#构建数据集
x1 = [1,1,1,1,1,2,2,2,2,2,3,3,3,3,3]
x2 = ['S','M','M','S','S','S','M','M','L','L','L','M','M','L','L']
y = [-1,-1,1,1,-1,-1,-1,1,1,1,1,1,1,1,-1]

df = pd.DataFrame({'x1':x1, 'x2':x2, 'y':y})
X = df[['x1','x2']]
y = df['y']

In [27]:
class NaiveBayes:
    def __init__(self):
        pass
    def fit(self,X,y):
        #类先验
        self.class_prior = pd.value_counts(y ,normalize=True)
        #类别标签
        self.class_labels = self.class_prior.index
        #条件先验
        self.prior = {}
        #该过程相当于存表
        for l in self.class_labels:
            for x in X.columns:
                px_given_y = X[x][(y == l).values].value_counts(normalize = True)
                for p in px_given_y.index:
                    self.prior[(l,x,p)] = px_given_y[p]
        return 
    def predict(self,X):
        res = []
        for c in self.class_labels:
            p_y = self.class_prior[c]
            pxy = 1
            #查表
            for x in X.items():
                pxy *= self.prior[tuple([c]+list(x))]
            res.append(pxy)
        return self.class_labels[np.argmax(res)]

In [28]:
model = NaiveBayes()
model.fit(X,y)
model.predict({'x1': 2, 'x2': 'S'})

-1